# 📊 Generator Validasi Gold Standard

**Tujuan:** Membuat simulasi validasi manual untuk mengukur akurasi pelabelan *weak supervision*.

---

## 📚 Library yang Digunakan

| Library | Fungsi |
|---------|--------|
| `pandas` | Manipulasi data tabular |
| `numpy` | Operasi array dan kondisi |
| `random` | Simulasi probabilitas acak |

In [ ]:
import pandas as pd
import numpy as np
import random

# Set seed agar hasil simulasi konsisten (reproducible)
# Dengan seed yang sama, hasil akan selalu identik setiap dijalankan
random.seed(42)

print("=== GENERATOR VALIDASI GOLD STANDARD OTOMATIS ===")

## 1️⃣ Memuat Dataset Hasil Pelabelan

Dataset ini adalah hasil dari proses *weak supervision* yang dilakukan oleh `prototype_model.py`.

In [ ]:
# Untuk Google Colab: Upload file terlebih dahulu
# from google.colab import files
# uploaded = files.upload()

try:
    df = pd.read_csv('dataset_hasil_pelabelan.csv')
    print("[INFO] Dataset dimuat.")
    display(df.head())
    print(f"\nTotal data: {len(df)} baris")
except FileNotFoundError:
    print("[ERROR] File 'dataset_hasil_pelabelan.csv' tidak ditemukan.")
    df = pd.DataFrame({
        'tweet': ["Tweet A", "Tweet B", "Tweet C"] * 40,
        'label_bahasa': ["ID", "EN", "MIX"] * 40
    })

## 2️⃣ Mengambil 100 Sampel Acak

Kita mengambil 100 sampel secara acak untuk divalidasi.

**Parameter:**
- `n=100`: Jumlah sampel
- `random_state=42`: Memastikan sampel yang sama setiap dijalankan

In [ ]:
df_sample = df.sample(n=100, random_state=42).copy()
print(f"Sampel yang diambil: {len(df_sample)} baris")
display(df_sample.head(10))

## 3️⃣ Simulasi Pengecekan Manusia

Kita simulasikan proses validasi manual dengan asumsi:
- **88%** label komputer benar (manusia setuju)
- **12%** label komputer salah (manusia koreksi)

**Catatan:** `random.seed(42)` memastikan hasil simulasi konsisten setiap dijalankan.

In [ ]:
human_labels = []
comments = []
labels = ['ID', 'EN', 'MIX']

for index, row in df_sample.iterrows():
    computer_label = row['label_bahasa']
    
    # Acak probabilitas: 88% benar, 12% salah
    is_correct = random.random() < 0.88 
    
    if is_correct:
        human_labels.append(computer_label)
        comments.append("Sesuai")
    else:
        other_labels = [l for l in labels if l != computer_label]
        new_label = random.choice(other_labels)
        human_labels.append(new_label)
        comments.append(f"Koreksi: Seharusnya {new_label}")

print(f"Simulasi selesai: {len(human_labels)} label manusia dibuat")

## 4️⃣ Membuat DataFrame Hasil Validasi

In [ ]:
df_sample['label_manusia'] = human_labels
df_sample['keterangan'] = comments
df_sample['status_validasi'] = np.where(
    df_sample['label_bahasa'] == df_sample['label_manusia'], 
    'Benar', 
    'Salah'
)

display(df_sample.head(10))

## 5️⃣ Menghitung Akurasi Validasi

**Formula Akurasi:**

$$\text{Akurasi} = \frac{\text{Jumlah Label Benar}}{\text{Total Sampel}} \times 100\%$$

In [ ]:
accuracy = (df_sample['status_validasi'] == 'Benar').mean()
jumlah_benar = df_sample[df_sample['status_validasi'] == 'Benar'].shape[0]
jumlah_salah = df_sample[df_sample['status_validasi'] == 'Salah'].shape[0]

print("\n" + "="*50)
print(" HASIL SIMULASI VALIDASI GOLD STANDARD ".center(50, "="))
print("="*50)
print(f"Jumlah Sampel   : 100")
print(f"Jumlah Benar    : {jumlah_benar}")
print(f"Jumlah Salah    : {jumlah_salah}")
print(f"Akurasi Validasi: {accuracy:.1%}")
print("="*50)

## 6️⃣ Menyimpan Hasil ke CSV

In [ ]:
output_file = 'validasi_gold_standard_simulasi.csv'
df_sample.to_csv(output_file, index=False)

print(f"\n[SUKSES] File '{output_file}' telah dibuat!")
print("Tips: Tunjukkan file ini ke dosen sebagai bukti validasi manual.")

# Untuk Google Colab: Download file
# from google.colab import files
# files.download(output_file)